# Init Vault

In [1]:
%%bash
gcloud auth login
eval "$(terraform output -raw vault_init_remote)"

# If you want to save time and get the token directly uncomment
echo "VAULT TOKEN: $VAULT_TOKEN"
echo "VAULT ADDR: $VAULT_ADDR"

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=ryvdnemPLpavwmHHDlW79EMWA7qHSi&access_type=offline&code_challenge=Bxak0Q57dnrOLWfcdatKGRUlxBuW2lxJQHDuEy0Zke4&code_challenge_method=S256


You are now logged in as [jose.merchan@hashicorp.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Key                      Value
---                      -----
Recovery Seal Type       gcpckms
Initialized              false
Sealed                   true
Total Recovery Shares    0
Threshold                0
Unseal Progress          0/0
Unseal Nonce             n/a
Version                  1.17.6+ent
Build Date               2024-09-24T19:48:48Z
Storage Type             raft
HA Enabled               true


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0    744      0 --:--:-- --:--:-- --:--:--   800


<a href="/ui/">Temporary Redirect</a>.



Created secret [root_token_europe-southwest1_ftn3].
Created version [1] of the secret [root_token_europe-southwest1_ftn3].
Created secret [vault_init_data_europe-southwest1_ftn3].
Created version [1] of the secret [vault_init_data_europe-southwest1_ftn3].
Command flags must be provided before positional arguments. The following arguments will not be parsed as flags: [-path=localfile/]


Success! Enabled the file audit device at: file/
Success! Enabled the file audit device at: stdout/


created key [9b6dc0711e8fbac9db1952648d0555f8ca57028b] of type [json] as [sa-keys__europe-southwest1_ftn3.json] for [europe-southwest1-savault-ftn3@hc-ef850f7ac0e04fa581e54b3fba8.iam.gserviceaccount.com]


Success! Data written to: sys/storage/raft/snapshot-auto/config/hourly
VAULT TOKEN: hvs.tWzFqz9vcbiOtGoba5OKJAEO
VAULT ADDR: https://vault-europe-southwest1-ftn3.josemerchan-2c4ef2.gcp.sbx.hashicorpdemo.com:8200


In [2]:
! terraform output

boundary_fqdn_443 = "https://boundary-europe-southwest1-s39l.josemerchan-2c4ef2.gcp.sbx.hashicorpdemo.com"
boundary_fqdn_9200 = "https://boundary-europe-southwest1-s39l.josemerchan-2c4ef2.gcp.sbx.hashicorpdemo.com:9200"
crypto_key = "kms-boundary-key-recovery-s39l"
crypto_key_worker = "kms-boundary-key-worker-auth-s39l"
helm = <sensitive>
key_ring = "kms-boundary-keyring-s39l"
kubernetes_cluster = <sensitive>
location = "global"
project = "hc-ef850f7ac0e04fa581e54b3fba8"
remove_database_before_destroy = "gcloud sql instances delete europe-southwest1-postgres-instance-s39l --project=hc-ef850f7ac0e04fa581e54b3fba8"
remove_peering_before_destroy = "gcloud compute networks peerings delete servicenetworking-googleapis-com --network=europe-southwest1-ss9a --project=hc-ef850f7ac0e04fa581e54b3fba8"
vault_fqdn_8200 = "https://vault-europe-southwest1-ftn3.josemerchan-2c4ef2.gcp.sbx.hashicorpdemo.com:8200"
vault_fqdn_8201 = "https://vault-clusterport-europe-southwest1-ftn3.josemerchan-2c4ef2.gcp.

# Init Boundary

In [3]:
%%bash
mkdir init_boundary
cat << EOF > ./init_boundary/main.tf 
# https://developer.hashicorp.com/boundary/docs/install-boundary/initialize#create-your-first-login-account
provider "boundary" {
  addr             = "$(terraform output -raw boundary_fqdn_443)"
  tls_insecure     = true
  recovery_kms_hcl = <<EOT
    kms "gcpckms" {
    purpose     = "recovery"
    key_ring    = "$(terraform output -raw key_ring)"
    crypto_key  = "$(terraform output -raw crypto_key)"
    project     = "$(terraform output -raw project)"
    region      = "$(terraform output -raw location)"
    }
EOT
}

resource "boundary_auth_method" "password" {
  name        = "Password auth method"
  description = "Password auth method"
  type        = "password"
  scope_id    = "global"
}

resource "boundary_account_password" "myuser" {
  name           = "admin"
  description    = "User account for adminr"
  login_name     = var.boundary_user
  password       = var.boundary_password
  auth_method_id = boundary_auth_method.password.id
}

resource "boundary_user" "myuser" {
  name        = "admin"
  description = "Initial Admin User"
  account_ids = [boundary_account_password.myuser.id]
  scope_id    = "global"
}

resource "boundary_role" "org_admin" {
  scope_id        = "global"
  grant_scope_ids = ["global"]
  grant_strings = [
    "ids=*;type=*;actions=*"
  ]
  principal_ids = [boundary_user.myuser.id]
}

output "auth_method" {
  value = boundary_auth_method.password.id
}

EOF

cat << EOF > ./init_boundary/variables.tf 
variable "boundary_user" {
  description = "Initial Boundary Admin User"
  type        = string
}

variable "boundary_password" {
  description = "Password for Boundary Admin"
  type        = string
}
EOF

cat << EOF > ./init_boundary/variables.tfvars
boundary_user        = "admin"            #Use your own
boundary_password    = "SuperSecret123!"  #Use your own
EOF

mkdir: init_boundary: File exists


In [4]:
%%bash
cd init_boundary
terraform init
terraform apply --auto-approve --var-file=variables.tfvars


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/boundary from the dependency lock file
- Using previously-installed hashicorp/boundary v1.1.15

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
boundary_auth_method.password: Refreshing state... [id=ampw_HEbVjxX920]
boundary_account_password.myuser: Refreshing state... [id=acctpw_pLXwRagE4R]
boundary_user.myuser: Refreshing state... [id=u_krUHOBRyZv]
boundary_role.org_admin: Refreshing state... [id=r_6ugxQY4ktZ]

Note: Objects have changed outside of Terraform

Terraform detected the following changes ma

#### Run in a terminal in directory ```GCP_Playground/4_Simple_Boundary_Demo/1_Infra_internal_vault_internal_boundary```
```bash
gcloud auth login
export VAULT_ADDR=$(terraform output -raw vault_fqdn_8200)
export BOUNDARY_ADDR=$(terraform output -raw boundary_fqdn_443)
export VAULT_SKIP_TLS=true
export VAULT_TOKEN=$($(eval "terraform output -raw vault_read_vault_token"); echo) #The extra echo is due to how zsh cd orks
export TF_VAR_vault_addr=$(echo $VAULT_ADDR)
export TF_VAR_authmethod=$(terraform output -state=./init_boundary/terraform.tfstate -raw auth_method)
boundary authenticate password -auth-method-id $(terraform output -state=./init_boundary/terraform.tfstate -raw auth_method)
```


## To destroy configuration

In [ ]:
%%bash
cd init_boundary
terraform destroy --auto-approve --var-file=variables.tfvars